In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
customer_data = pd.read_excel('d2c_analytics.xlsx');
df = customer_data.copy()
df.head()

,Order Date,Ship Date,Customer Name,State,Category,Segment,Sub-Category,Sales,Quantity,Discount,Profit
0,2012-01-01,2012-01-06,Toby Braunhardt,Rajasthan,Office Supplies,Consumer,Storage,408.300,2,0.0,106.140
1,2012-01-01,2012-01-08,Joseph Holt,Uttar Pradesh,Office Supplies,Consumer,Supplies,120.366,3,0.1,36.036
2,2012-01-01,2012-01-05,Annie Thurman,Haryana,Office Supplies,Consumer,Storage,66.120,4,0.0,29.640
3,2012-01-01,2012-01-05,Eugene Moren,Punjab,Office Supplies,Home Office,Paper,44.865,3,0.5,-26.055
4,2012-01-01,2012-01-08,Joseph Holt,Uttar Pradesh,Furniture,Consumer,Furnishings,113.670,5,0.1,37.770


In [3]:
df.shape

(51290, 11)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Order Date     51290 non-null  datetime64[ns]
 1   Ship Date      51290 non-null  datetime64[ns]
 2   Customer Name  51290 non-null  object        
 3   State          51290 non-null  object        
 4   Category       51290 non-null  object        
 5   Segment        51290 non-null  object        
 6   Sub-Category   51290 non-null  object        
 7   Sales          51290 non-null  float64       
 8   Quantity       51290 non-null  int64         
 9   Discount       51290 non-null  float64       
 10  Profit         51290 non-null  float64       
dtypes: datetime64[ns](2), float64(3), int64(1), object(5)
memory usage: 4.3+ MB


In [5]:
df.isnull().sum()

Order Date       0
Ship Date        0
Customer Name    0
State            0
Category         0
Segment          0
Sub-Category     0
Sales            0
Quantity         0
Discount         0
Profit           0
dtype: int64

In [6]:
RFM=pd.DataFrame(index=df['Customer Name'].unique())

In [7]:
normalize = lambda x : (x-x.min())/(x.max()-x.min())

Recency Analysis

In [8]:
day_max = df['Order Date'].max()

def recency(col,w = np.array([0.4,0.3,0.2,0.1])):
    recent_days = col.sort_values(ascending=False).head(4)
    vals = (w*(day_max - recent_days)).sum()
    return vals

customer_recency = df.pivot_table(values='Order Date',index='Customer Name',aggfunc=recency)
customer_recency['Recent_days'] = customer_recency.apply(lambda x : x.dt.days)
customer_recency.drop('Order Date',axis=1,inplace=True)
customer_recency

,Recent_days
Customer Name,
Aaron Bergman,45
Aaron Hawkins,48
Aaron Smayling,23
Adam Bellavance,39
Adam Hart,19
...,...
Xylona Preis,48
Yana Sorensen,41
Yoseph Carroll,27


In [9]:
recent_day_max = customer_recency['Recent_days'].max()
customer_recency['Inverse_recent_days'] = recent_day_max-customer_recency['Recent_days']
customer_recency['R_score'] = normalize(customer_recency['Inverse_recent_days'])
customer_recency.head()

,Recent_days,Inverse_recent_days,R_score
Customer Name,,,
Aaron Bergman,45,541,0.923208
Aaron Hawkins,48,538,0.918089
Aaron Smayling,23,563,0.960751
Adam Bellavance,39,547,0.933447
Adam Hart,19,567,0.967577


In [10]:
customer_recency['Inverse_recent_days'].describe

<bound method NDFrame.describe of Customer Name
Aaron Bergman         541
Aaron Hawkins         538
Aaron Smayling        563
Adam Bellavance       547
Adam Hart             567
                     ... 
Xylona Preis          538
Yana Sorensen         545
Yoseph Carroll        559
Zuschuss Carroll      573
Zuschuss Donatelli    585
Name: Inverse_recent_days, Length: 796, dtype: int64>

In [11]:
RFM['R_score'] = customer_recency['R_score']
RFM.head()

,R_score
Toby Braunhardt,0.889078
Joseph Holt,0.977816
Annie Thurman,0.996587
Eugene Moren,0.936860
Magdelene Morse,0.941980


Frequency Analysis

In [12]:
customer_frequency = pd.DataFrame(df['Customer Name'].value_counts())
customer_frequency.columns = ['Frequency']
customer_frequency['Normalized_frequency'] = normalize(customer_frequency['Frequency'])
customer_frequency.head()

,Frequency,Normalized_frequency
Muhammed Yedwab,108,1.000000
Steven Ward,106,0.980198
Patrick O'Brill,102,0.940594
Bill Eplett,102,0.940594
Gary Hwang,102,0.940594


In [13]:
customer_frequency['Entry_date'] = df[['Customer Name','Order Date']].groupby('Customer Name').min()
customer_frequency['N-days'] = (day_max - customer_frequency['Entry_date']).dt.days
customer_frequency['Normalized_days'] = normalize(customer_frequency['N-days'])
customer_frequency.head()

,Frequency,Normalized_frequency,Entry_date,N-days,Normalized_days
Muhammed Yedwab,108,1.000000,2012-01-14,1447,0.985126
Steven Ward,106,0.980198,2012-06-01,1308,0.826087
Patrick O'Brill,102,0.940594,2012-01-26,1435,0.971396
Bill Eplett,102,0.940594,2012-04-12,1358,0.883295
Gary Hwang,102,0.940594,2012-01-25,1436,0.972540


In [14]:
customer_frequency['Order/day'] = customer_frequency['Normalized_frequency']/customer_frequency['Normalized_days']
customer_frequency['F_score'] = normalize(customer_frequency['Order/day'])
customer_frequency.head()

,Frequency,Normalized_frequency,Entry_date,N-days,Normalized_days,Order/day,F_score
Muhammed Yedwab,108,1.000000,2012-01-14,1447,0.985126,1.015099,0.808593
Steven Ward,106,0.980198,2012-06-01,1308,0.826087,1.186555,0.991943
Patrick O'Brill,102,0.940594,2012-01-26,1435,0.971396,0.968291,0.758539
Bill Eplett,102,0.940594,2012-04-12,1358,0.883295,1.064869,0.861816
Gary Hwang,102,0.940594,2012-01-25,1436,0.972540,0.967152,0.757320


In [15]:
RFM['F_score'] = customer_frequency['F_score']
RFM.head()

,R_score,F_score
Toby Braunhardt,0.889078,0.358348
Joseph Holt,0.977816,0.548928
Annie Thurman,0.996587,0.231294
Eugene Moren,0.936860,0.644218
Magdelene Morse,0.941980,0.348475


Monetary Analysis

In [16]:
zscore = lambda x : (x-x.mean())/x.std()
customer_monetary=df[['Customer Name','Profit']].groupby('Customer Name').sum().astype('int')
customer_monetary.head()

,Profit
Customer Name,
Aaron Bergman,4683
Aaron Hawkins,2450
Aaron Smayling,369
Adam Bellavance,4979
Adam Hart,1902


In [17]:
customer_monetary.describe()

,Profit
count,796.000000
mean,1843.146985
std,1545.150089
min,-6151.000000
25%,983.750000
50%,1786.500000
75%,2602.500000
max,8672.000000


In [18]:
customer_monetary['z_score'] = zscore(customer_monetary['Profit'])

In [19]:
z_loss_score = (0-customer_monetary.z_score.mean())/customer_monetary.z_score.std()

def weigh_score(val,w1=3,w2=-4,w3=-3):
    if val>1:
        return(w1*val)
    elif val<z_loss_score:
        return(w2*val)
    elif val<-1:
        return(w3*val)
    return val

customer_monetary['M_score'] = normalize(customer_monetary['z_score'].apply(weigh_score))
customer_monetary.head()

,Profit,z_score,M_score
Customer Name,,,
Aaron Bergman,4683,1.837914,0.266326
Aaron Hawkins,2450,0.392747,0.018837
Aaron Smayling,369,-0.954048,0.184286
Adam Bellavance,4979,2.029481,0.294100
Adam Hart,1902,0.038089,0.001697


In [20]:
RFM['M_score'] = customer_monetary['M_score']
RFM.head()

,R_score,F_score,M_score
Toby Braunhardt,0.889078,0.358348,0.074440
Joseph Holt,0.977816,0.548928,0.182159
Annie Thurman,0.996587,0.231294,0.025374
Eugene Moren,0.936860,0.644218,0.166958
Magdelene Morse,0.941980,0.348475,0.237895


RFM Net Score

In [21]:
wr_rf,wf_rf=1,1
RFM['RF_score'] = normalize(wr_rf*RFM['R_score'] + wf_rf*RFM['F_score'])

wr_rm,wm_rm = 1,1
RFM['RM_score'] = normalize(wr_rm*RFM['R_score'] + wm_rm*RFM['M_score'])

wf_fm,wm_fm = 1,1
RFM['FM_score'] = normalize(wf_fm*RFM['F_score'] + wm_fm*RFM['M_score'])

RFM.head()

,R_score,F_score,M_score,RF_score,RM_score,FM_score
Toby Braunhardt,0.889078,0.358348,0.074440,0.557152,0.396721,0.288780
Joseph Holt,0.977816,0.548928,0.182159,0.723399,0.520437,0.511813
Annie Thurman,0.996587,0.231294,0.025374,0.545519,0.433524,0.157099
Eugene Moren,0.936860,0.644218,0.166958,0.755739,0.485073,0.571694
Magdelene Morse,0.941980,0.348475,0.237895,0.582762,0.532968,0.403611


In [22]:
w_r,w_f,w_m = 1,1,1
RFM['RFM_1_score'] = normalize(w_r*RFM['R_score'] + w_f*RFM['F_score'] + w_m*RFM['M_score']) 

w_rf,w_rm,w_fm  = 1,1,1
RFM['RFM_2_score'] = normalize(w_rf*RFM['RF_score'] + w_rm*RFM['RM_score'] + w_fm*RFM['FM_score'])

RFM.head()

,R_score,F_score,M_score,RF_score,RM_score,FM_score,RFM_1_score,RFM_2_score
Toby Braunhardt,0.889078,0.358348,0.074440,0.557152,0.396721,0.288780,0.417362,0.392838
Joseph Holt,0.977816,0.548928,0.182159,0.723399,0.520437,0.511813,0.650247,0.629923
Annie Thurman,0.996587,0.231294,0.025374,0.545519,0.433524,0.157099,0.376078,0.343614
Eugene Moren,0.936860,0.644218,0.166958,0.755739,0.485073,0.571694,0.673793,0.656199
Magdelene Morse,0.941980,0.348475,0.237895,0.582762,0.532968,0.403611,0.541606,0.520712


In [27]:
RFM.sort_values('RFM_1_score',ascending=False).head()

,R_score,F_score,M_score,RF_score,RM_score,FM_score,RFM_1_score,RFM_2_score
Bill Eplett,0.906143,0.861816,0.522206,0.866969,0.689443,1.000000,1.000000,1.000000
Tamara Chand,0.895904,0.723817,0.640622,0.778739,0.757566,0.985358,0.982056,0.983941
Harry Greene,0.979522,0.873676,0.266859,0.917703,0.574850,0.817949,0.897643,0.886351
Luke Foster,0.960751,0.471136,0.686351,0.666941,0.827199,0.830624,0.896549,0.892943
Tom Ashbrook,0.972696,0.693658,0.419085,0.806495,0.666414,0.797169,0.876813,0.867669


In [33]:
RFM = RFM.sort_values('RFM_2_score',ascending=False)
RFM.head()

,R_score,F_score,M_score,RF_score,RM_score,FM_score,RFM_1_score,RFM_2_score
Bill Eplett,0.906143,0.861816,0.522206,0.866969,0.689443,1.000000,1.000000,1.000000
Tamara Chand,0.895904,0.723817,0.640622,0.778739,0.757566,0.985358,0.982056,0.983941
Luke Foster,0.960751,0.471136,0.686351,0.666941,0.827199,0.830624,0.896549,0.892943
Harry Greene,0.979522,0.873676,0.266859,0.917703,0.574850,0.817949,0.897643,0.886351
Raymond Buch,0.909556,0.545634,0.620073,0.680811,0.753223,0.836770,0.870690,0.868004


In [34]:
RFM.to_excel('customer_RFM.xlsx')
